# Table of Contents
 <p><div class="lev2 toc-item"><a href="#bitrex-code-wip" data-toc-modified-id="bitrex-code-wip-01"><span class="toc-item-num">0.1&nbsp;&nbsp;</span>bitrex code wip</a></div>

In [ ]:
import json
import hmac
import time
import base64
import urllib3
import hashlib
import requests
import pandas as pd
from requests.auth import AuthBase

In [ ]:
api_cred = pd.read_json("../API_KEYS/api_keys.json", orient= "index")
api_url = pd.read_json("../API_KEYS/api_url.json", orient= "index")

In [ ]:
api_url_gdax = api_url.loc["GDAX"].URL
api_url_gdax

In [ ]:
# Create custom authentication for Exchange
class GDAXExchangeAuth(AuthBase):
    def __init__(self, exchange):
        self.api_key = api_cred.loc[exchange].KEY
        self.secret_key = api_cred.loc[exchange].SECRET
        self.passphrase = api_cred.loc[exchange].PASSPHRASE

    def __call__(self, request):
        timestamp = str(time.time())
        message = timestamp + request.method + request.path_url + (request.body or '')
        hmac_key = base64.b64decode(self.secret_key)
        signature = hmac.new(hmac_key, message.encode('utf-8'), hashlib.sha256)
        digest = signature.digest()
        signature_b64 = base64.b64encode(digest).decode('utf-8').rstrip('\n')

        request.headers.update({
            'CB-ACCESS-SIGN': signature_b64,
            'CB-ACCESS-TIMESTAMP': timestamp,
            'CB-ACCESS-KEY': self.api_key,
            'CB-ACCESS-PASSPHRASE': self.passphrase,
            'Content-Type': 'application/json'
        })
        return request

In [ ]:
auth = GDAXExchangeAuth(exchange="GDAX")
auth

In [ ]:
# Get accounts
balance_data = requests.get(api_url_gdax + 'accounts', auth=auth)
balance_data = balance_data.json()
balance_data = pd.DataFrame(balance_data)
balance_data

In [ ]:
for acct in balance_data.id:
    account = 'accounts/'+ acct +'/ledger'
    acct_hist = requests.get(api_url_gdax + account, auth=auth)
    temp = acct_hist.json()
    temp = pd.DataFrame(temp)
    temp.replace({r'\n': ''}, regex=True)
    print(temp)

## bitrex code wip

In [ ]:
api_url_btrx = api_url.loc["BTRX"].URL
api_url_btrx

In [ ]:
MARKET_SET = {
    'getopenorders',
    'cancel',
    'sellmarket',
    'selllimit',
    'buymarket',
    'buylimit'
}

ACCOUNT_SET = {
    'getbalances',
    'getbalance',
    'getdepositaddress',
    'withdraw',
    'getorderhistory',
    'getorder',
    'getdeposithistory',
    'getwithdrawalhistory'
}

BUY_ORDERBOOK = 'buy'
SELL_ORDERBOOK = 'sell'
BOTH_ORDERBOOK = 'both'

In [ ]:
def using_requests(request_url, apisign):
    return requests.get(
                request_url,
                headers={"apisign": apisign}
            ).json()

In [ ]:
class BTRXExchangeAuth(AuthBase):
    """
    Used for requesting Bittrex with API key and API secret
    """
    def __init__(self, exchange):
        self.api_key = api_cred.loc[exchange].KEY
        self.secret_key = api_cred.loc[exchange].SECRET
        self.passphrase = api_cred.loc[exchange].PASSPHRASE
#         self.dispatch = dispatch


#     def api_query(self, method):
#         """
#         Queries Bittrex with given method and options
#         :param method: Query method for getting info
#         :type method: str
#         :param options: Extra options for query
#         :type options: dict
#         :return: JSON response from Bittrex
#         :rtype : dict
#         """
#         nonce = str(int(time.time() * 1000))
#         method_set = 'public'

#         if method in MARKET_SET:
#             method_set = 'market'
#         elif method in ACCOUNT_SET:
#             method_set = 'account'

#         request_url = (api_url % method_set) + method + '?'

#         if method_set != 'public':
#             request_url += 'apikey=' + self.api_key + "&nonce=" + nonce + '&'

#         apisign = hmac.new(self.api_secret.encode(),
#                            request_url.encode(),
#                            hashlib.sha512).hexdigest()
#         return self.dispatch(request_url, apisign)

#     def get_markets(self):
#         """
#         Used to get the open and available trading markets
#         at Bittrex along with other meta data.
#         :return: Available market info in JSON
#         :rtype : dict
#         """
#         return self.api_query('getmarkets')
    
    def __call__(self, request):
#         nonce = str(int(time.time() * 1000))       
#         hmac_key = base64.b64decode(self.secret_key)
#         request_url_encode = base64.b64decode(request_url)
        timestamp = str(time.time())
        message = timestamp + request.method + request.path_url + (request.body or '')
        hmac_key = base64.b64decode(self.secret_key)
        signature = hmac.new(hmac_key, message.encode('utf-8'), hashlib.sha256)
        digest = signature.digest()
        signature_b64 = base64.b64encode(digest).decode('utf-8').rstrip('\n')
        
#         apisign = hmac.new(
#             hmac_key,
#             request_url_encode,
#             hashlib.sha512
#             ).hexdigest()
        apisign = signature_b64

        return apisign

In [ ]:
auth = BTRXExchangeAuth(exchange="BTRX")
auth

In [ ]:
api_url_btrx = api_url.loc["BTRX"].URL
api_url_btrx

In [ ]:
api_url = api_url_btrx + "market" + "/" + "getopenorders/"
api_url

In [ ]:
x = requests.get(api_url, auth)
x